In [ ]:
from openprompt.data_utils import InputExample

example = InputExample(
    guid=0,
    text_a="برطانیہ کے سابق فوجی سربراہ کا کہنا ہے",
    meta={
        "entity": "برطانیہ"
    },
    label="LOCATION"
)


In [ ]:
import pandas as pd
from openprompt.data_utils import InputExample

# ==============================
# Load the train CSV
# ==============================
train_df = pd.read_csv(r"C:\Users\stdFurqan\Desktop\paft\NER\ner_train.csv")

# ==============================
# Prepare labels and mapping
# ==============================
labels = train_df["label"].unique().tolist()
label_map = {label: idx for idx, label in enumerate(labels)}
classes = labels

# ==============================
# 16-shot sampling per label
# ==============================
guid = 0
examples = []
class_stats = {}

for label_name in classes:
    # Take FIRST 16 examples per class
    class_df = train_df[train_df["label"] == label_name].head(16)
    class_stats[label_name] = len(class_df)

    for _, row in class_df.iterrows():
        examples.append(
            InputExample(
                guid=guid,
                text_a=str(row["sentence"]).strip(),  # full sentence
                text_b=str(row["entity"]).strip(),    # the entity itself
                label=label_map[label_name]           # numeric label
            )
        )
        guid += 1

# ==============================
# Print InputExamples
# ==============================
print("\n### InputExamples ###\n")
for ex in examples:
    print(
        "InputExample(\n"
        f"    guid={ex.guid},\n"
        f"    text_a=\"{ex.text_a}\",\n"
        f"    text_b=\"{ex.text_b}\",\n"
        f"    label={ex.label}\n"
        "),"
    )

# ==============================
# Print Metadata
# ==============================
print("\n### Classes ###")
print(f"classes = {classes}")

print("\n### Label Map ###")
print(f"label_map = {label_map}")

print("\n### Samples per Class ###")
for cls, count in class_stats.items():
    print(f"{cls}: {count}")

print(f"\nTotal InputExamples generated: {guid}")


In [ ]:
templates = [
    ("P1", ManualTemplate(
        text='جملہ: {"placeholder":"text_a"} لفظ {"placeholder":"text_b"} ایک {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P2", ManualTemplate(
        text='{"placeholder":"text_a"} میں موجود لفظ {"placeholder":"text_b"} کی قسم {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P3", ManualTemplate(
        text='مندرجہ ذیل جملے میں: {"placeholder":"text_b"} ایک {"mask"} کے طور پر آتا ہے: {"placeholder":"text_a"}',
        tokenizer=tokenizer,
    )),

    ("P4", ManualTemplate(
        text='جملے: {"placeholder":"text_a"} لفظ {"placeholder":"text_b"} کا درجہ {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P5", ManualTemplate(
        text='{"placeholder":"text_a"} → لفظ {"placeholder":"text_b"} کی شناخت: {"mask"}',
        tokenizer=tokenizer,
    )),

    ("P6", ManualTemplate(
        text='جملہ میں {"placeholder":"text_b"} ایک {"mask"} قسم کا لفظ ہے: {"placeholder":"text_a"}',
        tokenizer=tokenizer,
    )),

    ("P7", ManualTemplate(
        text='یہ لفظ {"placeholder":"text_b"} جملے {"placeholder":"text_a"} میں {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P8", ManualTemplate(
        text='جملے {"placeholder":"text_a"} میں لفظ {"placeholder":"text_b"} کی قسم کیا ہے؟ {"mask"}',
        tokenizer=tokenizer,
    )),

    ("P9", ManualTemplate(
        text='جملہ: {"placeholder":"text_a"} لفظ {"placeholder":"text_b"} کو درجہ بندی کریں: {"mask"}',
        tokenizer=tokenizer,
    )),

    ("P10", ManualTemplate(
        text='{"placeholder":"text_a"} → {"placeholder":"text_b"} ایک {"mask"} کی مثال ہے۔',
        tokenizer=tokenizer,
    )),
]


In [1]:
# Set device
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# ==============================
# Imports
# ==============================
import torch
import pandas as pd
import random
import numpy as np
from collections import Counter
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader
from torch.optim import AdamW
from sklearn.metrics import classification_report
from collections import defaultdict
from torch.utils.data import DataLoader, Sampler

# ========================================
# Check CUDA
# ========================================
device = "cuda" #if torch.cuda.is_available() else "cpu"
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)
    print("GPU count:", torch.cuda.device_count())

# ========================================
# Seeds for reproducibility
# ========================================
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
GPU name: NVIDIA GeForce RTX 4080 SUPER
CUDA version: 12.1
GPU count: 1


In [3]:
# ==============================
# Balanced Batch Sampler
# ==============================
class BalancedBatchSampler(Sampler):
    def __init__(self, dataset, batch_size):
        """
        dataset: list of InputExample
        batch_size: total batch size (must be divisible by number of classes)
        """
        self.dataset = dataset
        self.labels = [ex.label for ex in dataset]
        self.classes = list(sorted(set(self.labels)))
        self.num_classes = len(self.classes)
        assert batch_size % self.num_classes == 0, "Batch size must be divisible by number of classes"
        self.batch_size_per_class = batch_size // self.num_classes

    def __iter__(self):
        class_indices = {c: np.where(np.array(self.labels) == c)[0].tolist() for c in self.classes}
        for c in self.classes:
            np.random.shuffle(class_indices[c])

        num_batches = min(len(class_indices[c]) // self.batch_size_per_class for c in self.classes)

        for i in range(num_batches):
            batch = []
            for c in self.classes:
                start = i * self.batch_size_per_class
                end = start + self.batch_size_per_class
                batch.extend(class_indices[c][start:end])
            np.random.shuffle(batch)
            yield batch

    def __len__(self):
        return min(len(np.where(np.array(self.labels) == c)[0]) // self.batch_size_per_class for c in self.classes)

In [4]:

# Step 1: Training data (16-shot) ha,m and spam
train_dataset = [

        InputExample(
            guid=0,
            text_a="یہ بھی کہا گیا ہے کہ موت کی سزا کا استعمال ہمیشہ سے ہی نفرت انگیز رہا ہے لیکن ایران میں وحشیانہ طور پر مقدمات غیر منصفانہ انداز میں چلائے جاتے ہیں۔",
            text_b="ایران",
            label=0
        ),
        InputExample(
            guid=1,
            text_a="بھارت نے اس دورۂ زمبابوے میں 3 ون ڈے انٹرنیشنل اور دو ٹی ٹوئنٹی میچ کھیلنے ہیں۔",
            text_b="بھارت",
            label=0
        ),
        InputExample(
            guid=2,
            text_a="دو دن پہلے چین کی جانب سے تصدیق موصول ہوئی ہے کہ ان منصوبوں پر دستخط چین ہی میں کیے جا سکتے ہیں۔",
            text_b="چین",
            label=0
        ),
        InputExample(
            guid=3,
            text_a="انھوں نے کہا کہ سنگین جرائم میں ملوث 90 فیصد ملزمان کو سزائیں ہو رہی ہیں جبکہ غیر قانون اسلحے سمیت دیگر چھوٹے مقدمات میں انھیں ضمانت مل جاتی ہے اس میں جسٹس سسٹم میں کچھ خامیاں ہیں اور کچھ تفتیش میں بھی کمی رہی جاتی ہے جس کو بہتر بنانے کی ضرورت ہے۔﻿پشاور میں بڈا پیر میں پاکستان فضائیہ کی بیس پر حملے سے قبل بھی کراچی، کوئٹہ، کامرہ اور پشاور میں پاکستان فضائیہ کی ایئر بیس اور ایئرپورٹس کو نشانہ بنایا گیا تھا۔",
            text_b="کامرہ",
            label=0
        ),
        InputExample(
            guid=4,
            text_a="واضح رہے ڈھاکہ میں کھیلے گئے پہلے ایک روزہ میچ میں جنوبی افریقہ نے آٹھ وکٹوں سے کامیابی حاصل کی تھی جبکہ ڈھاکہ میں کھیلے گئے دوسرے میچ میں بنگلہ دیش فاتح رہا تھا۔﻿پریماداسا کرکٹ سٹیڈیم میں کھیلے جانے والے پانچ ایک روز میچوں کی سیریز کے تیسرے میچ میں پاکستان نے سری لنکا کو 135 رنز سے شکست دے کر سیریز میں دو ایک سے برتری حاصل کر لی ہے۔",
            text_b="پاکستان",
            label=0
        ),
        InputExample(
            guid=5,
            text_a="ان کا کہنا تھا کہ پنجاب رینجرز نے سات ارب روپے سے زیادہ اور بلوچستان نے ایک ارب ساٹھ کروڑ روپے سے زائد رقم خرچ کی ہے۔",
            text_b="بلوچستان",
            label=0
        ),
        InputExample(
            guid=6,
            text_a="10دسمبر2007 کو پاکستان میں جنگی جہاز سازی کے مرکز کامرہ میں واقع ادارے پاکستان ایرو ناٹیکل کپملیکس کے فوجی گیٹ کے قریب پاکستان فضائیہ کی بس میں خودکش حملہ کیا گیا تھا جس کے نتیجے میں سات افراد ہلاک ہوئے۔",
            text_b="پاکستان",
            label=0
        ),
        InputExample(
            guid=7,
            text_a="تیونس میں آنے والی تبدیلی تھوڑے ہی عرصے پورے خطے کو اپنی لپیٹ میں لے لیا اور مصر لیبیا یمن اور شام میں ایسی ہی تحریکوں نے جنم لیا۔",
            text_b="یمن",
            label=0
        ),
        InputExample(
            guid=8,
            text_a="ان کا یہ ریکارڈ اس وقت ٹوٹتے ٹوٹتے ہی رہ گیا تھا جب پاکستان کے خلاف ویسٹ انڈیز کے آل راؤنڈر آندرے رسل نے 13 گیندوں پر 42 رنز کی اننگز کھیلی۔",
            text_b="پاکستان",
            label=0
        ),
        InputExample(
            guid=9,
            text_a="تاہم بعد میں جورووٹ گیری بیلنس اور بین سٹوکس کی شاندار بیٹنگ کی بدولت انگلینڈ کی پوزیشن مستحکم ہوگئی۔",
            text_b="انگلینڈ",
            label=0
        ),
        InputExample(
            guid=10,
            text_a="بھارت نے اس دورۂ زمبابوے میں 3 ون ڈے انٹرنیشنل اور دو ٹی ٹوئنٹی میچ کھیلنے ہیں۔",
            text_b="زمبابوے",
            label=0
        ),
        InputExample(
            guid=11,
            text_a="الشباب حال ہی میں باراوی سمیت کئی قصبوں پر کنٹرول کھو چکی ہے لیکن دیہاتی علاقوں میں اب بھی بہت سے علاقے ان کے قبضے میں ہیں۔",
            text_b="باراوی",
            label=0
        ),
        InputExample(
            guid=12,
            text_a="ان کا کہنا تھا کہ ’تمام تر مظالم کے باوجود میں یقین دلاتا ہوں کہ اگر بھارت یا را نے پاکستان کے خلاف کوئی منصوبہ بندی کی اور پاکستانی فوج کو بھارت سے لڑنا پڑا تو ایم کیوایم کے کارکنان، پاکستان کے دفاع کے لیے پاک فوج کے شانہ بشانہ لڑنے کے لیے تیار ہیں۔",
            text_b="پاکستان",
            label=0
        ),
        InputExample(
            guid=13,
            text_a="محمد حفیظ ہمسٹرنگ کی تکلیف کے سبب دبئی ٹیسٹ نہیں کھیلےتھے۔",
            text_b="دبئی",
            label=0
        ),
        InputExample(
            guid=14,
            text_a="سے سنہ دو ہزارنو تک برطانوی فوج کے سنہ دوہزار چھ سربراہ لارڈ ڈانٹ کا کہنا ہے کہ ’شاید مجھے اس بات کا اندازہ لگا لینا چاہیے تھا کہ عراق کے حالات بہتر نہیں ہوں گے اور سنہ دوہزارچھ میں عراق میں برطانوی فوجیوں کی تعداد پندرہ سو تک کم کر دینا درست فیصلہ نہیں تھا۔",
            text_b="عراق",
            label=0
        ),
        InputExample(
            guid=15,
            text_a="مارچ میں تیونس کے دارالحکومت تیونس میں ایک عجائب گھر پر ہونے والے حملے میں کم از کم 22 افراد مارے گئے تھے۔",
            text_b="تیونس",
            label=0
        ),
        InputExample(
            guid=16,
            text_a="لیکن اس کے بعد یونس خان اور مصباح الحق کے درمیان 125 رنز کی شراکت نے سری لنکا کے ہاتھ کچھ بھی نہیں رہنے دیا۔",
            text_b="125",
            label=1
        ),
        InputExample(
            guid=17,
            text_a="ایک سال قبل ان کی ذاتی دولت کا تخمینہ دس اعشاریہ دو ارب ڈالر لگایا گیا تھا جو اب بڑھ کر چودہ اعشاریہ چار ارب ڈالر ہو گئی ہے۔",
            text_b="دس اعشاریہ دو ارب",
            label=1
        ),
        InputExample(
            guid=18,
            text_a="24جولائی 2013 میں سکھر میں آئی ایس آئی کے دفتر پر حملے اور بم دھماکوں میں آٹھ افراد ہلاک ہوئے جبکہ چار حملہ آور جوابی کارروائی کے دوران مارے گئے، کراچی کے بعد اندرون سندھ میں اپنی نوعیت کا یہ پہلا حملہ تھا۔﻿پاکستانی فوج کے ترجمان کا کہنا ہے کہ صوبہ خیبرپختونخوا کے دارالحکومت پشاور کے نواح میں واقع پاکستانی فضائیہ کے کیمپ پر حملے میں 43 افراد ہلاک ہوئے ہیں۔",
            text_b="چار",
            label=1
        ),
        InputExample(
            guid=19,
            text_a="جمعرات کو پاکستان ے چار وکٹوں کے نقصان پر 219 رنز سے اپنی اننگز دوبارہ شروع کی اور کپتان مصباح الحق 69 رنزبنا کر آؤٹ ہو گئے جس کے بعد وکٹ کیپر سرفراز اور اسد شفیق کے درمیان شاندار 124 رنز کی شراکت ہوئی۔",
            text_b="124",
            label=1
        ),
        InputExample(
            guid=20,
            text_a="اس سلسلے میں احکام بھی جاری کردیے گئے ہیں۔﻿کولمبو ٹیسٹ کے دوسرے روز سری لنکن ٹیم پاکستان کے 138 رنز کے جواب میں اپنی پہلی اننگز میں 315 رنز بنا کر آؤٹ ہو گئی ہے۔",
            text_b="138",
            label=1
        ),
        InputExample(
            guid=21,
            text_a="چوتھی وکٹ کے لیے اظہر علی اور اسد شفیق کے درمیان 46 رنز کی شراکت ہوئی جس کا خاتمہ اسد شفیق کے 15 رنز پر آؤٹ ہونے کے بعد ہوا۔",
            text_b="15",
            label=1
        ),
        InputExample(
            guid=22,
            text_a="دوسرے دن انگلینڈ کی جانب سے معین علی اور سٹوارٹ براڈ نے 7 وکٹوں کے نقصان پر 343 رنز سے اننگز دوبارہ شروع کی تو اس کی بقیہ تین وکٹیں سکور میں 87 رنز کے اضافے کے بعد گریں۔",
            text_b="7",
            label=1
        ),
        InputExample(
            guid=23,
            text_a="وراٹ كوہلی کی کپتانی میں بھارتی ٹیم نے تین ٹیسٹ میچوں کی سیریز 1 2 سے اپنے نام کی۔",
            text_b="2",
            label=1
        ),
        InputExample(
            guid=24,
            text_a="سندھ حکومت کے خلاف درخواست جمع کرانے کے بعد میڈیا سے بات کرتے ہوئے عمران اسماعیل کا کہنا تھا کہ 2 ہزار شہری سندھ حکومت کی مجرمانہ غفلت پرجان سے ہاتھ دھوبیٹھے جس کے خلاف صوبائی حکومت کے خلاف مقدمے کے لئے درخواست عمران خان کے میڈیا ایڈوائزر کے طور پردرج کرائی ہے جب کہ درخواست میں وزیراعلی سندھ سید قائم علی شاہ سمیت وزرا کو نامزد کیا گیا ہے اوراگر 72 گھنٹے میں ایف آئی آرنہ کاٹی گئی توتھانے کے باہردھرنا دیں گے۔",
            text_b="2 ہزار",
            label=1
        ),
        InputExample(
            guid=25,
            text_a="اس میچ کی ایک خاص بات اور بھی ہے کہ اس میں فٹبال کے عالمی کپ میں سب سے زیادہ گول کرنے والے کھلاڑی کے علاوہ تاریخ کے پانچ میں سے چار مہنگے ترین کھلاڑی بھی حصہ لیں گے۔",
            text_b="پانچ",
            label=1
        ),
        InputExample(
            guid=26,
            text_a="قلیل اور طویل مدت کے منصوبے لگانے کا اس سے زیادہ اچھا موقع آپ کو دنیا میں کہیں اور نہیں مل سکتا جہاں ہم آپ سے پانچ ہزار میگاواٹ بجلی خریدنے کی تحریری ضمانت دینے کو تیار ہیں۔",
            text_b="پانچ ہزار",
            label=1
        ),
        InputExample(
            guid=27,
            text_a="اس سے پہلے بھارت 6 وکٹ پر 317 رنز اسکور کیے، شیکھر دھون اور دھونی نے ففٹیز بنائیں۔",
            text_b="6",
            label=1
        ),
        InputExample(
            guid=28,
            text_a="مارٹن گپٹل 42 ،  ٹام لیتھم 16، کین ولیمسن 90، کولن مونرو 9، گرینٹ ایلیٹ 36 اور لیوک رونچی 7 رنز بنا کر آؤٹ ہوئے جب کہ جیمس نیشم 37 اور نیتھن میکیولم 25 رنز بنا کر ناٹ آؤٹ رہے۔",
            text_b="36",
            label=1
        ),
        InputExample(
            guid=29,
            text_a="پاکستانی ٹیم کی میچ پر گرفت اس وقت مضبوط دکھائی دینے لگی تھی جب اس نے کوشل پریرا کے آتش فشاں کو ٹھنڈا کرنے کے بعد صرف 19 رنز کے اضافے پر چار وکٹیں اپنے قابو میں کر لی تھیں لیکن دنیش چندی مل کے ذمہ دارانہ 48 رنز نے میزبان ٹیم کو 49 ویں اوور میں جیت سے ہمکنار کر دیا۔",
            text_b="48",
            label=1
        ),
        InputExample(
            guid=30,
            text_a="چین کا گوگل کہلانے والی کمپنی بائیدو کے مالک رابن لی چودہ اعشاریہ سات ارب ڈالر کے اثاثوں کے ساتھ دوسرے نمبر پر ہیں۔",
            text_b="چودہ اعشاریہ سات",
            label=1
        ),
        InputExample(
            guid=31,
            text_a="اس سے قبل پاکستان نے گال میں کھیلے گئے سیریز کے پہلے ٹیسٹ میچ میں میزبان ٹیم کو دس وکٹوں سے شکست دت تھی جبکہ کولمبو میں کھیلے گئے دوسرے ٹیسٹ میچ میں سری لنکا نے پاکستان کو سات وکٹوں سے ہرا دیا تھا۔﻿انٹرنیشنل کرکٹ کونسل نے بھارتی سابق کرکٹر سچن تندولکر کو ورلڈ کپ سنہ 2015 کا سفیر مقرر کر دیا ہے۔",
            text_b="دس",
            label=1
        ),
        InputExample(
            guid=32,
            text_a="سفیر کی حیثیت سے سچن تندولکر کے فرائض میں آئی سی سی کرکٹ ورلڈ کپ سنہ 2015 کے دوران آئی سی سی کے اقدامات اور اس ایونٹ کو فروغ دینا شامل ہو گا۔",
            text_b="سنہ 2015",
            label=2
        ),
        InputExample(
            guid=33,
            text_a="اس سیریز میں بھارت اور زمبابوے کے درمیان تین ون ڈے انٹرنیشنل اور دو ٹی ٹوئنٹی میچ ہوں گے جس کی دس جولائی سے ابتدا ہو رہی ہے۔",
            text_b="دس جولائی",
            label=2
        ),
        InputExample(
            guid=34,
            text_a="تنظیم کا کہنا ہے کہ ایک اندازے کے مطابق اس سال 151 افراد کی سزائے موت پر عملدرآمد کیا گیا ہے جو کہ 1995 سنہ سے اب تک سب سے زیادہ تعداد ہے۔",
            text_b="1995 سنہ",
            label=2
        ),
        InputExample(
            guid=35,
            text_a="سعید انور نے اپنے کیریئر میں 47.1 کی اوسط سے رنز بنائے تھے۔﻿سری لنکا کے معروف بیٹسمین اور وکٹ کیپر کمار سنگا کارا رواں برس اگست میں ٹیسٹ کرکٹ سے ریٹائر ہو جائیں گے۔",
            text_b="اگست",
            label=2
        ),
        InputExample(
            guid=36,
            text_a="﻿﻿سعودی عرب کی خواتین گاڑی چلانے پر پابندی کے خلاف مہم کی پہلی برسی کے موقعے پر چھبیس اکتوبر ایک مرتبہ پھر گاڑی چلانے کا حق دیے جانے کا مطالبہ دہرایا گیا۔",
            text_b="چھبیس اکتوبر",
            label=2
        ),
        InputExample(
            guid=37,
            text_a="لارڈ رچرڈ افغانستان میں برطانوی فوجی بھیجنے کے بعد برطانیہ کے فوجی حکام کا خیال تھا کہ سنہ دوہزارپانچ تک عراق کی صورت حال میں بہتری آئے گی لیکن ان کا یہ اندازہ بھی غلط ثابت ہوا۔",
            text_b="سنہ دوہزارپانچ",
            label=2
        ),
        InputExample(
            guid=38,
            text_a="اقوامِ متحدہ کے بچوں کے ادارے یونیسیف کے مطابق جہاں پاکستان میں سنہ دوہزارچودہ میں پولیو کے ریکارڈ مریض سامنے آئے ہیں وہیں نائجیریا اور افغانستان میں اس مرض کے پھیلاؤ میں کمی دیکھی گئی ہے۔",
            text_b="سنہ دوہزارچودہ",
            label=2
        ),
        InputExample(
            guid=39,
            text_a="﻿﻿یورپ کے بڑے فٹبال کلب فٹ بال کی عالمی تنظیم فیفا سے یہ درخواست کرنے پر غور کر رہے ہیں کہ وہ سنہ دو ہزارباءیس میں قطر میں ہونے والا فٹبال ورلڈ کپ کا انعقاد مئی کے مہینے میں کیا جائے۔",
            text_b="سنہ دو ہزارباءیس",
            label=2
        ),
        InputExample(
            guid=40,
            text_a="اس سے قبل چار جنوری کو شمالی وزیرستان کی ہی تحصیل دتہ خیل میں الوڑہ منڈی کے قریب کالعدم تنظیم تحریک طالبان کے ایک دھڑے کے مرکز پر حملےمیں سات پسند مارے گئے تھے۔",
            text_b="چار جنوری",
            label=2
        ),
        InputExample(
            guid=41,
            text_a="سنہ دوہزارچودہ کے آغاز پر اقوام متحدہ کے ادارہ برائے صحت نے پولیو کے مرض کے پھیلاؤ کی وجہ سے پاکستانیوں پر بیرون ملک سفر کے حوالے سے پابندیاں عائد کی تھیں۔",
            text_b="سنہ دوہزارچودہ",
            label=2
        ),
        InputExample(
            guid=42,
            text_a="رپورٹ کے مطابق کمیشن اس نتیجے پر پہنچا ہے کہ سنہ 2013 کے عام انتخابات قانون اور آئین کے مطابق ہوئے اور ان کے نتائج عوامی مینڈیٹ کا عکاس ہیں۔",
            text_b="سنہ 2013",
            label=2
        ),
        InputExample(
            guid=43,
            text_a="پاکستان کرکٹ بورڈ چاہتا ہے کہ آئی سی سی ان کی سزا میں نرمی کر دے تاکہ وہ پاکستان میں کھیلنے کے ساتھ ساتھ پاکستان کرکٹ بورڈ کی تربیتی سہولتوں سے فائدہ اٹھا سکیں تاکہ جب اگست دوہزارپندرہ میں جب ان پر پابندی ختم ہو تو وہ بین الاقوامی کرکٹ کے لیے تیار ہوں۔",
            text_b="اگست دوہزارپندرہ",
            label=2
        ),
        InputExample(
            guid=44,
            text_a="آسٹریلیا کی ٹیم اس سے پہلے سنہ 2013 میں بھارت کے خلاف سپن کے لیے سازگار حالات میں چار صفر اور سنہ 2014 میں متحدہ عرب امارات میں پاکستان کے خلاف مردہ وکٹوں پر تین صفر سے سیریز ہار چکی ہے اور ان دونوں سیریز میں آسٹریلوی ٹیم اس بات کو سمجھنے سے قاصر رہی کہ وہاں کیا حکمتِ عملی اپنائی جائے؟۔",
            text_b="سنہ 2014",
            label=2
        ),
        InputExample(
            guid=45,
            text_a="دوسری جانب سوات میں سنہ دوہزارتیرہ کی نسبت سنہ دوہزارچودہ میں جرائم کی شرح میں تیس فیصد اضافہ ہوا ہے جس میں سرکاری اعداد و شمار کےمطابق اب تک قتل کے انتہر ، اقدام قتل کے چون ، دہشت گردی کے پندرہ اور ٹارگٹ کلنگ کے نو واقعات رپورٹ ہوئے ہیں۔",
            text_b="سنہ دوہزارتیرہ",
            label=2
        ),
        InputExample(
            guid=46,
            text_a="میں حقوقِ انسانی کے لیے سرگرم ادارے ہیومن رائٹس کونسل آف پاکستان کے مطابق سنہ1999 سے سنہ 2012 تک تیرہ برس کے عرصے میں آٹھ سو شیعہ ہزارہ ہلاک کیے گئے جبکہ سنہ2013 کے ابتدائی چند ہفتوں میں ٹارگٹ کلنگ اور بم دھماکوں میں دو سو سے زائد افراد کی جان گئی۔",
            text_b="سنہ1999",
            label=2
        ),
        InputExample(
            guid=47,
            text_a="فیفا نے چالیس سے زیادہ درجہ حرارت میں کھیلنے والے کھلاڑیوں کی صحت کے متعلق بڑھتی ہوئی تشویش کے بعد گذشتہ اکتوبر کو مزید صلاح مشورے کے عمل کا اعلان کیا تھا۔",
            text_b="اکتوبر",
            label=2
        ),
        InputExample(
            guid=48,
            text_a="سری لنکا کے شہر کولمبو میں کھیلے جا رہے میچ میں کھیل کے تیسرے روز سری لنکا نے اپنی پہلی نا مکمل اننگز 301 رنز پر شروع کی لیکن 14 رنز کے اضافے کے بعد دشمنتھا چیمارا رنز بنا کر یاسر شاہ کا شکار ہو گئے جب کہ رنگنا ہیراتھ 18 رنز بنا کر ناٹ آؤٹ رہے۔",
            text_b="یاسر شاہ",
            label=3
        ),
        InputExample(
            guid=49,
            text_a="پولیس کے تفتیشی افسر منیر خان نے بتایا کہ حملے کے وقت دفتر میں دوست محمد خان کے بھتیجے اور دو رشتہ دار موجود تھے۔",
            text_b="منیر خان",
            label=3
        ),
        InputExample(
            guid=50,
            text_a="بدخشاں میں پولیس کے صوبائی ترجمان  لال محمد احمدزئی نے بی بی سی کو بتایا کہ جھڑپ کے دوران طالبان کا بھی بھاری جانی نقصان ہوا ہے۔",
            text_b="لال محمد احمدزئی",
            label=3
        ),
        InputExample(
            guid=51,
            text_a="فرانسیسی خبر رساں ایجنسی اے ایف پی کے مطابق عینی شاہد بابمی الحاجی کولو نے بتایا کہ بدھ کو 50 کے قریب شدت پسندوں نے ککوا گاؤں پر حملہ کیا۔",
            text_b="بابمی الحاجی کولو",
            label=3
        ),
        InputExample(
            guid=52,
            text_a="ڈیو رچرڈسن نے کہا کہ قوانین سب کے لیے یکساں ہیں اور قومیت سے قطع نظر کوئی بھی بولر مشکوک بولنگ ایکشن کی زد میں آیا اسے رپورٹ کیا جائے گا۔",
            text_b="ڈیو رچرڈسن",
            label=3
        ),
        InputExample(
            guid=53,
            text_a="تفصیلات کے مطابق بنگلہ دیشی بیٹسمین 318 رنز کے تعاقب میں جانے والی اپنی  ٹیم کو مضبوط شراکتیں فراہم نہیں کرپائے، تمیم اقبال صرف 5 رنز پر آئوٹ ہوئے، صابر رحمان نے سب سے زیادہ 43 اور سومیہ سرکار نے 40 رنز بنائے، پوری ٹیم 47 اوورز میں 240 رنز پر رخصت ہوئی۔",
            text_b="صابر رحمان",
            label=3
        ),
        InputExample(
            guid=54,
            text_a="ریحانہ جباری کا موقف رہا تھا کہ انھوں نے جنسی استحصال کی کوشش کرنے پر اپنے دفاع میں قتل کیا۔",
            text_b="ریحانہ جباری",
            label=3
        ),
        InputExample(
            guid=55,
            text_a="ان کا ساتھ محمود اللہ نے دیا جنھوں نے بھی نصف سنچری بنائی۔",
            text_b="محمود اللہ",
            label=3
        ),
        InputExample(
            guid=56,
            text_a="آئی پی ایل اور اس کے بعد بنگلہ دیش میں اچھی کارکردگی کا مظاہرہ کرنے والے بیٹسمین اجنکیا رہانے کو ٹیم انڈیا کا کپتان مقرر کیا گیا ہے۔",
            text_b="اجنکیا رہانے",
            label=3
        ),
        InputExample(
            guid=57,
            text_a="مشتاق مہر کے مطابق گزشتہ دو سالوں میں پولیس نے 6112 ملزمان کو گرفتار کیا جن میں 1798 قتل ، 803 دہشت گردی، 121 اغوا، 534 بھتہ خوری، 2856 ڈکیتی کی واردات میں ملوث تھے ۔",
            text_b="مشتاق مہر",
            label=3
        ),
        InputExample(
            guid=58,
            text_a="بی بی سی کے نامہ نگار کا کہنا ہے کہ ڈیوڈ کیمرون کی جماعت کے ایک رکن نے انھیں بتایا ہے کہ ایک شخص کو اپنی طرف آتا دیکھ کر وزیرِ اعظم پیچھے ہٹ گئے اور ان کا اس سے کوئی رابطہ نہیں ہوا۔",
            text_b="وزیرِ اعظم",
            label=3
        ),
        InputExample(
            guid=59,
            text_a="قیادت اظہر علی کے سپرد ہوگی، سلیکٹرز نے احمد شہزاد محمد حفیظ شعیب ملک عمراکمل بابر اعظم محمد رضوان سرفراز احمد جنید خان اسد شفیق عماد وسیم انور علی عمران خان بلاول بھٹی یاسر شاہ حماد اعظم سمیع اسلم مختار احمد احسان عادل راحت علی سعد نسیم کے ناموں پر غور کیا۔",
            text_b="احمد شہزاد",
            label=3
        ),
        InputExample(
            guid=60,
            text_a="تحریک انصاف کے چیرمین عمران خان نے کہا کہ عام انتخابات میں اس وقت کے چیف الیکشن کمشنر فخر الدین جی ابراہیم کو کچھ نہیں پتا تھا۔",
            text_b="عمران خان",
            label=3
        ),
        InputExample(
            guid=61,
            text_a="کپتان ایلسٹر کک کی سربراہی میں انگلینڈ کی ٹیم میں معین علی، جوس بٹلر، ایڈم لائیتھ، مارک وڈ اور عادل رشید ایسے کھلاڑی ہیں جن کی یہ پہلی ایشز سیریز ہو گی۔",
            text_b="جوس بٹلر",
            label=3
        ),
        InputExample(
            guid=62,
            text_a="مارٹن گپٹل 42 ،  ٹام لیتھم 16، کین ولیمسن 90، کولن مونرو 9، گرینٹ ایلیٹ 36 اور لیوک رونچی 7 رنز بنا کر آؤٹ ہوئے جب کہ جیمس نیشم 37 اور نیتھن میکیولم 25 رنز بنا کر ناٹ آؤٹ رہے۔",
            text_b="گرینٹ ایلیٹ",
            label=3
        ),
        InputExample(
            guid=63,
            text_a="اظہرعلی 64 اور یونس خان 23 رنز کے ساتھ کریز پر موجود ہیں۔",
            text_b="یونس خان",
            label=3
        ),
        InputExample(
            guid=64,
            text_a="ایس ایچ او تھانہ شالکوٹ جہانگیر نے بی بی سی سے بات کرتے ہوئے بتایا کہ فائرنگ کے نتیجے میں 8 افراد ہلاک اور ایک شخص زخمی ہو گیا۔",
            text_b="ایس ایچ او",
            label=4
        ),
        InputExample(
            guid=65,
            text_a="﻿﻿ملکۂ برطانیہ الزبتھ نے ایک ٹویٹ کے ساتھ لندن سائنس میوزیم کے اب تک کی سب سے بڑی نمائش کا افتتاح کیا۔",
            text_b="ملکۂ",
            label=4
        ),
        InputExample(
            guid=66,
            text_a="﻿﻿ترجمان وزیر اعظم ہاؤس نے واضح کیا ہے کہ وزیراعظم کی ایم کیو ایم کے وفد سے نئے صوبوں کے بارے میں کوئی بات نہیں ہوئی۔",
            text_b="ترجمان",
            label=4
        ),
        InputExample(
            guid=67,
            text_a="پیرس میں ملاقات کے دوران فرانس کے صدر فرانسوا اولاند اور جرمن چانسلر اینگلا میرکل نے یورپی سرحدوں پر کنٹرول سے متعلق بھی تبادلہ خیال کیا۔",
            text_b="صدر",
            label=4
        ),
        InputExample(
            guid=68,
            text_a="کپتان سمیت ون ڈے ٹیم میں شامل کئی کھلاڑی پہلے ہی سری لنکا میں موجود ہیں، دیگر منتخب پلیئرز 5 روزہ کیمپ میں نماز تراویح کے بعد ٹریننگ کیا کرینگے،رمضان المبارک کی وجہ سے کیمپ کا شیڈول معمول سے ہٹ کر ہوگا، رات 10بجے پریکٹس کے بعد بجے2 تک فزیکل سیشن بھی کیا جائے گا۔",
            text_b="کپتان",
            label=4
        ),
        InputExample(
            guid=69,
            text_a="ان بیوروکریٹس میں فیڈرل بورڈ آف ریونیو کے سابق چیئرمین عبداللہ یوسف ، ارشد حکیم اور سلمان صدیق کے علاوہ پانی و بجلی کے سابق سیکریٹری شاہد رفیع اور اسماعیل قریشی شامل ہیں۔",
            text_b="سیکریٹری",
            label=4
        ),
        InputExample(
            guid=70,
            text_a="جب کہ اپوزیشن کی جانب سے اجمل میاں اور رانا بھگوان داس اور جسٹس ریٹائرڈ طارق پرویز کے نام پیش کئے گئے ہیں۔",
            text_b="جسٹس",
            label=4
        ),
        InputExample(
            guid=71,
            text_a="پنجاب یونیورسٹی سنڈیکیٹ کے اجلاس میں سابق وزیراعظم سید یوسف رضا گیلانی کے بیٹے عبدالقادر گیلانی کی گریجویشن کی ڈگری کے معاملے کا دوبارہ جائزہ لیا گیا۔",
            text_b="وزیراعظم",
            label=4
        ),
        InputExample(
            guid=72,
            text_a="تحریک انصاف کے دونوں امیدواروں نے دونوں حلقوں کے انتخابی نتائج کو الیکشن ٹریبیونل میں چیلنج کر رکھا تھا اور ڈھائی برس کی سماعت کے بعد حال ہی میں ان دونوں ٹریبیونلز نے تحریک انصاف کے امیدواروں کے حق میں فیصلہ دیا تھا۔﻿جمعرات کے روز وزیراعظم نواز شریف کی صدارت میں مسلم لیگ ن کے مشاورتی اجلاس کے بارے میں صحافیوں کو بریفنگ دیتے ہوئے وزیراطلاعات پرویز رشید نے کہا کہ حکومت نے دونوں فیصلوں کو قانونی عدالت کے بجائے عوامی عدالت کے سامنے لے جانے کا فیصلہ کیا ہے۔",
            text_b="وزیراطلاعات",
            label=4
        ),
        InputExample(
            guid=73,
            text_a="بھٹو سنہ انیس سواکتہر سے سنہ انیس سوستتر تک سوال چیف مارشل ایڈمنسٹریٹر صدر اور پھر وزیراعظم کی حیثیت سے برسرِ اقتدار رہے اور قتل کے ایک مقدمے میں چار اپریل انیس سواناسی کو پھانسی پر چڑھا دیے گئے۔",
            text_b="سوال چیف مارشل ایڈمنسٹریٹر",
            label=4
        ),
        InputExample(
            guid=74,
            text_a="تیونس کے وزیر اعظم حبیب الصید نے دولت اسلامیہ کی جانب سے حملے کی ذمہ داری قبول کیے جانے کے بعد ملک میں سکیورٹی کے سخت اقدامات کا اعلان کیا۔",
            text_b="وزیر اعظم",
            label=4
        ),
        InputExample(
            guid=75,
            text_a="بھارت کے سابق سفارتکار اور اقوام متحدہ کے نائب سیکرٹری جنرل چنمے گریخان کو ان سے کئی بار ملنے کا موقع ملا تھا۔",
            text_b="نائب سیکرٹری جنرل",
            label=4
        ),
        InputExample(
            guid=76,
            text_a="لیکن جب اُنھوں نے عام انتخابات میں دھاندلی کے الزامات پر قومی اسمبلی کے چار حلقے کھولنے سے متعلق پاکستان تحریک انصاف کی درخواست کو ترجیحی بنیادوں پر سننے سے انکار کردیا اور افتخار چوہدری کی ریٹائر ہو گئے تو پاکستان تحریک انصاف کی قیادت نے منظم دھاندلی کے تمام تر الزامات خفیہ ادارے کے بریگیڈیر کے بجائے سابق چیف جسٹس پر دھر دیے۔",
            text_b="چیف جسٹس",
            label=4
        ),
        InputExample(
            guid=77,
            text_a="کرنل وارن کا کہنا تھا جہاں تک دولتِ اسلامیہ کے لیے منصوبہ بندی کرنے والے بیرونی افراد کا تعلق ہے تو امریکی فوج انھیں بہت جلد تلاش کر کے انھیں بھی مار دے گی۔",
            text_b="کرنل",
            label=4
        ),
        InputExample(
            guid=78,
            text_a="بدخشاں میں پولیس کے صوبائی ترجمان  لال محمد احمدزئی نے بی بی سی کو بتایا کہ جھڑپ کے دوران طالبان کا بھی بھاری جانی نقصان ہوا ہے۔",
            text_b="صوبائی ترجمان",
            label=4
        ),
        InputExample(
            guid=79,
            text_a="عمران خان نے کہا کہ انتخابات میں 35 پنکچرز کی بات محض سیاسی تھی، بریگیڈیئر سیمسن نے ہمیں اور مرتضیٰ پویا نے امریکی سفیر سے 35 پنکچر سے متعلق بات کی جو ٹوئٹ ہوگیا۔",
            text_b="بریگیڈیئر",
            label=4
        ),
        InputExample(
            guid=80,
            text_a="سنگاکارا نے 15 سالہ کریئر میں جو ریکارڈز قائم کیے وہ بذات خود ان کی عظمت کا ثبوت فراہم کرتے ہیں۔",
            text_b="15 سالہ",
            label=5
        ),
        InputExample(
            guid=81,
            text_a="اس خلائی مہم کا مقصد ساڑھے چار ارب سال قبل نظام شمسی کی تشکیل کے بارے میں چند سربستہ رازوں سے پردہ اٹھانا ہے اور سائنس دانوں کو امید ہے کہ دمدار ستارے کی سطح کے تجزیے سے اس عمل میں مدد ملے گی۔",
            text_b="ساڑھے چار ارب سال",
            label=5
        ),
        InputExample(
            guid=82,
            text_a="﻿امریکی قیادت میں اتحادی افواج گذشتہ ایک سال سے شام اور عراق میں دولتِ اسلامیہ کے شدت پسندوں کو نشانہ بنا رہی ہیں جبکہ حال میں روس نے بھی شام میں دولتِ اسلامیہ کے ٹھکانوں پر بمباری کی ہے۔",
            text_b="ایک سال",
            label=5
        ),
        InputExample(
            guid=83,
            text_a="قبضوں کے حوالے سے پاکستان پوسٹ نے مکمل معلومات قومی اسمبلی کی قائمہ کمیٹی کو 3 ماہ قبل پیش کیں لیکن قبضہ مافیا کے خلاف کارروائی تاحال عمل میں نہیں لائی جا سکی۔",
            text_b="3 ماہ",
            label=5
        ),
        InputExample(
            guid=84,
            text_a="کپتان سمیت ون ڈے ٹیم میں شامل کئی کھلاڑی پہلے ہی سری لنکا میں موجود ہیں، دیگر منتخب پلیئرز 5 روزہ کیمپ میں نماز تراویح کے بعد ٹریننگ کیا کرینگے،رمضان المبارک کی وجہ سے کیمپ کا شیڈول معمول سے ہٹ کر ہوگا، رات 10بجے پریکٹس کے بعد بجے2 تک فزیکل سیشن بھی کیا جائے گا۔",
            text_b="بجے2",
            label=5
        ),
        InputExample(
            guid=85,
            text_a="بیرسٹر سلطان محمود چوہدری نے کہا کہ ٹریفالگر سکور پر ڈیڑھ بجے دن اجتماع ہوگا جس کے بعد ساڑھے تین بجے مارچ ٹین ڈاؤنگ سٹریٹ کی طرف روانہ ہو جائے گا۔",
            text_b="ساڑھے تین بجے",
            label=5
        ),
        InputExample(
            guid=86,
            text_a="سندھ حکومت کے خلاف درخواست جمع کرانے کے بعد میڈیا سے بات کرتے ہوئے عمران اسماعیل کا کہنا تھا کہ 2 ہزار شہری سندھ حکومت کی مجرمانہ غفلت پرجان سے ہاتھ دھوبیٹھے جس کے خلاف صوبائی حکومت کے خلاف مقدمے کے لئے درخواست عمران خان کے میڈیا ایڈوائزر کے طور پردرج کرائی ہے جب کہ درخواست میں وزیراعلی سندھ سید قائم علی شاہ سمیت وزرا کو نامزد کیا گیا ہے اوراگر 72 گھنٹے میں ایف آئی آرنہ کاٹی گئی توتھانے کے باہردھرنا دیں گے۔",
            text_b="72 گھنٹے",
            label=5
        ),
        InputExample(
            guid=87,
            text_a="پانچ ماہ قبل کرد جنگجوؤں نے امریکہ اتحادی افواج کی معاونت سے اس علاقے پر کنٹرول حاصل کیا تھا۔",
            text_b="پانچ ماہ",
            label=5
        ),
        InputExample(
            guid=88,
            text_a="ذرائع کے مطابق یہ ملاقات صبح گیارہ بجے ہوگی جس میں ڈاکٹر فاروق ستار سمیت ایم کیوایم کے دیگر رہنما بھی شریک ہوں گے۔",
            text_b="گیارہ بجے",
            label=5
        ),
        InputExample(
            guid=89,
            text_a="گزشتہ دو سالوں میں رینجرز نے ہزار6 سے زائد آپریشن کیے، جن میں ساڑھے آٹھ ہزار کے قریب گرفتاریاں عمل میں لائی گئیں، جن میں 913 دہشت گرد، 500 ٹارگٹ کلرز اور 347 بھتہ خور شامل ہیں ان گرفتار ملزمان میں سے 3500 کو پولیس کے حوالے کیاگیا۔",
            text_b="دو سالوں",
            label=5
        ),
        InputExample(
            guid=90,
            text_a="انہوں نے کہا کہ 12 گھنٹے کی لوڈ شیڈنگ کو اس سال 6 گھنٹے پر لائے ہیں اور ملک میں کہیں غیراعلانیہ لوڈشیڈنگ نہیں ہورہی صرف نادہندہ علاقوں میں بجلی بند کی جاتی ہے جب کہ تمام کرپٹ ملازمین کو پکڑلیا گیا اب چور صارفین بھی قوم پر رحم کریں۔",
            text_b="12 گھنٹے",
            label=5
        ),
        InputExample(
            guid=91,
            text_a="ان دونوں جماعتوں کے درمیان طے پانے والے معاہدے میں طے پایا تھا کہ اگر عدالتی کمیشن نے کہا کہ ان انتخابات میں منظم دھاندلی ہوئی ہے تو پھر وزیراعظم قومی اسمبلی تحلیل کردیں گے اور ایسا نہ ہونے کی صورت میں پاکستان تحریک انصاف انتخابی اصلاحات کی کمیٹی میں شامل ہوکر موثر کردار ادا کرے گی۔﻿وفاقی﻿ وزیر داخلہ چوہدری نثارعلی خان نے قومی اسمبلی کو بتایا کہ گذشتہ پانچ سال سال کے دوران دہشت گردی کے خلاف جنگ میں اڑتالیس ارب روپے سے زائد خرچ ہوئے ہیں۔",
            text_b="پانچ سال",
            label=5
        ),
        InputExample(
            guid=92,
            text_a="ان کا یہ بھی کہنا تھا کہ وہ چار ماہ تک حکومت کے خلاف پی ٹی آئی کے سربراہ عمران خان کے دھرنے پر ایک کتاب لکھ سکتے ہیں تاہم اس کا کوئی فائدہ نہیں، ہمیں آگے دیکھنا چاہیے۔﻿کراچی میں پاکستان پیپلز پارٹی کے سابق وفاقی وزیر ڈاکٹر عاصم کو رینجرز نے 90 روز کے لیے حراست میں لے لیا ہے۔",
            text_b="چار ماہ",
            label=5
        ),
        InputExample(
            guid=93,
            text_a="جنھیں روشنی کی رفتار سے سفر کرتے ہوئے زمین تک پہنچنے میں ستایس منٹ لگتے ہیں۔",
            text_b="ستایس منٹ",
            label=5
        ),
        InputExample(
            guid=94,
            text_a="خیال رہے کہ پاکستان تحریک انصاف نے گذشتہ برس 14 اگست کو انتخابی دھاندلیوں کو بنیاد بنا کر اسلام آباد میں پارلیمان کے سامنے دھرنا دیا تھا جو تقریباً چار ماہ جاری رہا تھا۔",
            text_b="چار ماہ",
            label=5
        ),
        InputExample(
            guid=95,
            text_a="بنوں کے تھانہ صدر کے پولیس اہلکار نے بتایا قتل کیے گئِ نوجوانوں کے نام اعزاز اور فہد بتائے گئے ہیں اور ان دونوں کی عمریں بائیس سے پچیس سال کے درمیان بتائی گئی ہیں۔",
            text_b="پچیس سال",
            label=5
        ),
        InputExample(
            guid=96,
            text_a="ایک سوال پر جب وزیر اعلی سے پوچھا گیا کہ اگر میاں نواز شریف اور وہ کسی قسم کے قرض نادہندہ نہیں ہیں تو پھر نیب نے اُن کے نام سپریم کورٹ میں جمع کرائی گئی فہرست میں کیوں ڈالے؟",
            text_b="سپریم کورٹ",
            label=6
        ),
        InputExample(
            guid=97,
            text_a="سرکاری ذرائع کا کہنا ہے کہ خط میں بی بی سی کی بدھ کے روز نشر ہونے والی تحقیقاتی رپورٹ کا حوالہ دیتے ہوئے کہا گیا ہے کہ حکومت پاکستان ان حقائق تک رسائی چاہتی ہے جو بی بی سی کی رپورٹ میں بیان کیے گئے ہیں۔",
            text_b="بی بی سی",
            label=6
        ),
        InputExample(
            guid=98,
            text_a="سنہ انیس سو باییس میں بی بی سی کے پہلے ریڈیو پروگرام کو نشر کرنے میں شامل آلات کے علاوہ سر ٹم برنرز لی کا کمپیوٹرنکسٹ ہے جس نے پہلی ویب سائٹ ہوسٹ کی تھی۔﻿﻿فوربز میگزین نے ای کامرس کمپنی علی بابا کے بانی جیک ما کو چین کا امیر ترین شخص قرار دیا ہے جن کے اثاثوں کی مالیت انیس اعشاریہ پانچ ارب ڈالر بتائی گئی ہے۔",
            text_b="علی بابا",
            label=6
        ),
        InputExample(
            guid=99,
            text_a="البتہ تجزیہ نگاروں کےمطابق اسلام پسند حركت النہضہ اور ان کے لبرل اتحادی نداء تیونس کے جیتنے کے امکانات زیادہ ہیں۔",
            text_b="نداء تیونس",
            label=6
        ),
        InputExample(
            guid=100,
            text_a="ریحانہ کی والدہ شعلہ پاکروان نے بی بی سی فارسی سروس سے بات کرتے ہوئے پھانسی کی تصدیق کرتے ہوئے کہا کہ وہ اپنی بیٹی کی لاش دیکھنے جا رہی ہیں۔",
            text_b="بی بی سی فارسی",
            label=6
        ),
        InputExample(
            guid=101,
            text_a="اسلام آباد میں پارلیمنٹ ہاؤس کے باہر میڈیا سے بات کرتے ہوئے ایاز صادق نے کہا کہ تحریک انصاف کے تینتیس میں سے پچیس اراکین قومی اسمبلی آئے اور میں نے صبح گیارہ بجے سے چاربجے تک ان کا چیمبر میں انتظار کیا کیونکہ مجھے استعفوں کے اصل ہونے اور کسی سیاسی دباؤ کے تحت استعفے دیئے جانے کی تصدیق کرنا ہے۔",
            text_b="تحریک انصاف",
            label=6
        ),
        InputExample(
            guid=102,
            text_a="﻿﻿چھ ماہ سے دنیا ان دوسو سے زائد لڑکیوں کا انتظار کر رہی ہے جنھیں نائجیریا کے شدت پسند گروہ بوکو حرام نے اغوا کیا تھا۔",
            text_b="بوکو حرام",
            label=6
        ),
        InputExample(
            guid=103,
            text_a="اس دوران فرانس کے بیشتر ارکان پارلیمان نے شام میں دولت اسلامیہ پر جنوری کے اوائل کے بعد بھی بمباری جاری رکھنے کے حق میں ووٹ دیا ہے۔",
            text_b="دولت اسلامیہ",
            label=6
        ),
        InputExample(
            guid=104,
            text_a="آئی ایس پی آر کا کہنا ہے کہ ایک گروہ نے کیمپ میں قائم مسجد میں داخل ہو کر 16 نمازیوں اور قریب ہی واقع بیرک میں موجود سات افراد کو ہلاک کیا اور ان سب ہلاک شدگان کا تعلق پاکستانی فضائیہ سے ہے۔",
            text_b="پاکستانی فضائیہ",
            label=6
        ),
        InputExample(
            guid=105,
            text_a="خیال رہے کہ یہ پریس کانفرنس ایک ایسے وقت میں کی ہے جب ایک دن پہلے ہی نیب نے سپریم کورٹ زیر تفتیش مقدمات کی ایک فہرست جمع کروائی ہے جس میں وزیراعظم میاں نواز شریف اور شہباز شریف کا نام بھی شامل ہے۔",
            text_b="نیب",
            label=6
        ),
        InputExample(
            guid=106,
            text_a="گلے دن یہ تقریر دنیا کے ہر بڑے اخبار کی سرخی بنی اور اس تقریر کے نو دن کے اندر اندر اقوام متحدہ نے اپنی قرارداد نمبر تین ہزاردوسوسینتیس منظور کر کے پی ایل او کو اقوام متحدہ میں مبصر کا درجہ دے دیا۔",
            text_b="اقوام متحدہ",
            label=6
        ),
        InputExample(
            guid=107,
            text_a="پاکستان فیڈرل یونین آف جرنلٹس کے سیکریٹری جنرل خورشید عباسی کی جانب سے جاری ہونے والے بیان میں ایم کیو ایم کے سربراہ الطاف حسین کو چوبیس گھنٹے کی ڈیڈ لائن دیتے ہوئے کہا گیا: صحافتی برادری سے فوری معافی نہ مانگی اور آئندہ ایسا رویہ اختیار نہ کرنے کی یقین دہانی نہ کروائی تو ان الطاف حسین کے خلاف پورے ملک میں احتجاجی تحریک شروع کی جائے گی۔",
            text_b="ایم کیو ایم",
            label=6
        ),
        InputExample(
            guid=108,
            text_a="اُنھوں نے کہا کہ عدالتی کمیشن کا فیصلہ پاکستان تحریک انصاف کے لیے ایک بہت بڑا دھچکا ہے اور لوگوں کو پی ٹی آئی پر اعتماد بحال کرنے میں ایک عرصہ درکار ہوگا۔",
            text_b="پی ٹی آئی",
            label=6
        ),
        InputExample(
            guid=109,
            text_a="ان کا کہنا تھا کہ یہ دیکھیں کہ بدلا کیا ہے نہ صرف پیرس پر حملہ ہوا ہے بلکہ دنیا اکٹھی ہو گئی ہے اور اقوامِ متحدہ کی سلامتی کونسل کی قرارداد پر متفق ہے۔",
            text_b="اقوامِ متحدہ",
            label=6
        ),
        InputExample(
            guid=110,
            text_a="انھوں نے آرمی چیف جنرل راحیل شریف ، ڈی جی آئی ایس آئی رضوان اختر اور ڈی جی رینجرز میجر جنرل بلال کو مخاطب کرتے ہوئے کہا کہ ’آپ میری باتوں کاغلط مطلب نہ لیں، ہمیں فوج کا دشمن نہ سمجھیں۔",
            text_b="آئی ایس آئی",
            label=6
        ),
        InputExample(
            guid=111,
            text_a="حکومت پاکستان نے اس سال ستمبر میں اس دورے کے التوا کے بعد دھرنا دینے والی جماعتوں، عمران خان کی پاکستان تحریک انصاف اور طاہر القادری کی پاکستان عوامی تحریک پر سخت تنقید کی تھی اور انھیں چونتیس ارب ڈالر کے منصوبوں سے ملک کو محروم کر دینے کا ذمہ دار ٹھہرایا تھا۔",
            text_b="پاکستان تحریک انصاف",
            label=6
        ),
        
    
]


In [5]:
### Classes ###
classes = ['LOCATION', 'NUMBER', 'DATE', 'PERSON', 'DESIGNATION', 'TIME', 'ORGANIZATION']

### Label Map ###
label_map = {'LOCATION': 0, 'NUMBER': 1, 'DATE': 2, 'PERSON': 3, 'DESIGNATION': 4, 'TIME': 5, 'ORGANIZATION': 6}



# # # Step 1: Use load_plm with 'roberta' to get the correct WrapperClass
# _, _, _, WrapperClass = load_plm("roberta", "roberta-base")  # Just to get the wrapper

# # # # Step 2: Manually load XLM-RoBERTa model/tokenizer
# model_name = "xlm-roberta-base"
# tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
# plm = XLMRobertaForMaskedLM.from_pretrained(model_name)

# ==============================
# Load Pretrained Language Model (mBERT)
# ==============================
plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-multilingual-cased")

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized fr

In [6]:
# ==============================
# Define Prompt Template (Manual)
# ==============================
template = ManualTemplate(
    text = 'جملہ: {"placeholder":"text_a"} لفظ {"placeholder":"text_b"} ایک {"mask"} ہے۔',
    tokenizer=tokenizer,
)




templates = [
    ("P1", ManualTemplate(
        text='جملہ: {"placeholder":"text_a"} لفظ {"placeholder":"text_b"} ایک {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P2", ManualTemplate(
        text='{"placeholder":"text_a"} میں موجود لفظ {"placeholder":"text_b"} کی قسم {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P3", ManualTemplate(
        text='مندرجہ ذیل جملے میں: {"placeholder":"text_b"} ایک {"mask"} کے طور پر آتا ہے: {"placeholder":"text_a"}',
        tokenizer=tokenizer,
    )),

    ("P4", ManualTemplate(
        text='جملے: {"placeholder":"text_a"} لفظ {"placeholder":"text_b"} کا درجہ {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P5", ManualTemplate(
        text='{"placeholder":"text_a"} → لفظ {"placeholder":"text_b"} کی شناخت: {"mask"}',
        tokenizer=tokenizer,
    )),

    ("P6", ManualTemplate(
        text='جملہ میں {"placeholder":"text_b"} ایک {"mask"} قسم کا لفظ ہے: {"placeholder":"text_a"}',
        tokenizer=tokenizer,
    )),

    ("P7", ManualTemplate(
        text='یہ لفظ {"placeholder":"text_b"} جملے {"placeholder":"text_a"} میں {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("P8", ManualTemplate(
        text='جملے {"placeholder":"text_a"} میں لفظ {"placeholder":"text_b"} کی قسم کیا ہے؟ {"mask"}',
        tokenizer=tokenizer,
    )),

    ("P9", ManualTemplate(
        text='جملہ: {"placeholder":"text_a"} لفظ {"placeholder":"text_b"} کو درجہ بندی کریں: {"mask"}',
        tokenizer=tokenizer,
    )),

    ("P10", ManualTemplate(
        text='{"placeholder":"text_a"} → {"placeholder":"text_b"} ایک {"mask"} کی مثال ہے۔',
        tokenizer=tokenizer,
    )),
]







verbalizer = ManualVerbalizer(
    classes=classes,
    label_words={
    'LOCATION': ['شہر', 'ملک'],           # City, Country
    'NUMBER': ['عدد', 'نمبر'],            # Number, Digit
    'DATE': ['تاریخ', 'دن'],              # Date, Day
    'PERSON': ['شخص', 'انسان'],           # Person, Human
    'DESIGNATION': ['عہدہ', 'منصب'],      # Designation, Position
    'TIME': ['وقت', 'گھنٹہ'],             # Time, Hour
    'ORGANIZATION': ['ادارہ', 'کمپنی'],     # Organization, Company
    },
    tokenizer=tokenizer,
)


In [7]:
# ==============================
# Create Prompt Model
# ==============================
prompt_model = PromptForClassification(
    template=template,
    plm=plm,
    verbalizer=verbalizer
)

# ==============================
# Training loop with BalancedBatchSampler + random template switching
# ==============================
T = 20   # epochs
K = 1    # steps per prompt
batch_size = 28





prompt_model.train()
optimizer = AdamW(prompt_model.parameters(), lr=1e-5)
all_epoch_patterns = {}

for epoch in range(T):
    print(f"\n🟦 Epoch {epoch+1}/{T}")

    # Random initial template
    prompt_name, current_template = random.choice(templates)
    epoch_pattern = []

    # Create PromptDataLoader with BalancedBatchSampler
    sampler = BalancedBatchSampler(train_dataset, batch_size=batch_size)
    train_loader = PromptDataLoader(
        dataset=train_dataset,
        tokenizer=tokenizer,
        template=current_template,
        tokenizer_wrapper_class=WrapperClass,
        max_seq_length=128,
        batch_size=batch_size,
        batch_sampler=sampler,
        shuffle=False  # shuffle is ignored when batch_sampler is used
    )

    step_counter = 0
    epoch_loss = 0.0

    for batch in train_loader:
        # Move batch to device
        # batch = {k: v.to(device) for k, v in batch.items()}
    
        optimizer.zero_grad()
        logits = prompt_model(batch)
        loss = torch.nn.CrossEntropyLoss()(logits, batch['label'])
        # loss = criterion(logits, batch["label"])
        
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_pattern.append(prompt_name)

        step_counter += 1

        # Switch template every K steps
        if step_counter % K == 0:
            prompt_name, current_template = random.choice(templates)

            # Rebuild PromptDataLoader with new template but same sampler
            train_loader = PromptDataLoader(
                dataset=train_dataset,
                tokenizer=tokenizer,
                template=current_template,
                tokenizer_wrapper_class=WrapperClass,
                max_seq_length=128,
                batch_size=batch_size,
                batch_sampler=sampler,
                shuffle=False
            )

    all_epoch_patterns[f"epoch_{epoch+1}"] = epoch_pattern
    print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}")
    print(f"Prompt pattern: {epoch_pattern}")


🟦 Epoch 1/20


tokenizing: 112it [00:00, 436.38it/s]
tokenizing: 112it [00:00, 506.85it/s]
tokenizing: 112it [00:00, 972.35it/s]
tokenizing: 112it [00:00, 563.24it/s]
tokenizing: 112it [00:00, 810.13it/s]


Epoch 1 Loss: 11.6482
Prompt pattern: ['P2', 'P1', 'P5', 'P4']

🟦 Epoch 2/20


tokenizing: 112it [00:00, 909.82it/s]
tokenizing: 112it [00:00, 627.56it/s]
tokenizing: 112it [00:00, 908.36it/s]
tokenizing: 112it [00:00, 880.07it/s]
tokenizing: 112it [00:00, 932.91it/s]


Epoch 2 Loss: 7.3900
Prompt pattern: ['P3', 'P2', 'P9', 'P2']

🟦 Epoch 3/20


...

Epoch 18 Loss: 0.7836
Prompt pattern: ['P1', 'P6', 'P7', 'P5']

🟦 Epoch 19/20


tokenizing: 112it [00:00, 539.88it/s]
tokenizing: 112it [00:00, 816.17it/s]
tokenizing: 112it [00:00, 884.33it/s]
tokenizing: 112it [00:00, 824.33it/s]
tokenizing: 112it [00:00, 890.94it/s]


Epoch 19 Loss: 0.5733
Prompt pattern: ['P4', 'P10', 'P6', 'P4']

🟦 Epoch 20/20


tokenizing: 112it [00:00, 866.54it/s]
tokenizing: 112it [00:00, 584.11it/s]
tokenizing: 112it [00:00, 566.25it/s]
tokenizing: 112it [00:00, 977.39it/s] 
tokenizing: 112it [00:00, 456.53it/s]

Epoch 20 Loss: 2.0350
Prompt pattern: ['P7', 'P8', 'P3', 'P5']


In [8]:
# ==============================
# Load NER Test Dataset
# ==============================
df = pd.read_csv(r"C:\Users\stdFurqan\Desktop\paft\NER\ner_test.csv")

# ==============================
# Convert to InputExamples
# ==============================
# Note: For NER, label is per entity, so sentence is text_a, entity is in meta
eval_dataset = [
    InputExample(
        guid=i,
        text_a=row['sentence'],
        meta={'entity': row['entity']},
        label=label_map[row['label']]
    )
    for i, row in df.iterrows()
]

# ==============================
# PromptDataLoader
# ==============================
eval_loader = PromptDataLoader(
    dataset=eval_dataset,
    tokenizer=tokenizer,
    template=template,              # your ManualTemplate for NER
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=8,
    shuffle=False
)

# ==============================
# Evaluate Prompt-based NER Model
# ==============================
prompt_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in eval_loader:
        logits = prompt_model(batch)                     # [batch_size, num_classes]
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(batch['label'].cpu().tolist())

# ==============================
# Print NER Classification Report
# ==============================
print("\n📊 NER Urdu Test Classification Report:")
print(classification_report(
    all_labels, 
    all_preds, 
    target_names=classes,   # ['LOCATION','NUMBER','DATE','PERSON','DESIGNATION','TIME','ORGANIZATION']
    digits=4
))

tokenizing: 932it [00:00, 1020.02it/s]



📊 NER Urdu Test Classification Report:
              precision    recall  f1-score   support

    LOCATION     0.4066    0.3045    0.3482       243
      NUMBER     0.3526    0.5829    0.4394       199
        DATE     0.1667    0.0732    0.1017        41
      PERSON     0.4444    0.2149    0.2897       242
 DESIGNATION     0.1875    0.3684    0.2485        57
        TIME     0.0357    0.0667    0.0465        15
ORGANIZATION     0.3082    0.3333    0.3203       135

    accuracy                         0.3348       932
   macro avg     0.2717    0.2777    0.2563       932
weighted avg     0.3607    0.3348    0.3266       932



In [ ]:
robeerrta

📊 NER Urdu Test Classification Report:
              precision    recall  f1-score   support

    LOCATION     0.5579    0.2181    0.3136       243
      NUMBER     0.4444    0.4020    0.4222       199
        DATE     0.1667    0.0976    0.1231        41
      PERSON     0.5000    0.0372    0.0692       242
 DESIGNATION     0.0979    0.5614    0.1667        57
        TIME     0.3333    0.2000    0.2500        15
ORGANIZATION     0.2616    0.5407    0.3527       135

    accuracy                         0.2725       932
   macro avg     0.3374    0.2939    0.2425       932
weighted avg     0.4268    0.2725    0.2606       932